In [4]:
! python main.py E:/data/voc07/VOCdevkit/

C:\Users\Max\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\container.py:16: UserWarning: nn.Container is deprecated. All of it's functionality is now implemented in nn.Module. Subclass that instead.
  warnings.warn("nn.Container is deprecated. All of it's functionality "
Traceback (most recent call last):
  File "main.py", line 166, in <module>
    train(train_loader, model, optimizer, epoch)
  File "main.py", line 80, in train
    for i, (im, gt) in (enumerate(train_loader)):
  File "C:\Users\Max\Anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\dataloader.py", line 615, in __next__
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "C:\Users\Max\Anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\dataloader.py", line 615, in <listcomp>
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "E:\condaDev\Faster-RCNN\examples\voc.py", line 110, in __getitem__
    target = self.target_transform(target)
  File "E:\condaDev\Fast

In [1]:
import torch.nn as nn
from roi_pooling import roi_pooling as _roi_pooling

from rpn import RPN as _RPN
from faster_rcnn import FasterRCNN as _FasterRCNN

class _Features(nn.Container):
  def __init__(self):
    super(_Features, self).__init__()
    self.m = nn.Conv2d(3, 3, 3, 16, 1)

  def forward(self, x):
    return self.m(x)

class _Classifier(nn.Container):
  def __init__(self):
    super(_Classifier, self).__init__()
    self.m1 = nn.Linear(3*7*7, 21)
    self.m2 = nn.Linear(3*7*7, 21*4)

  def forward(self, x):
    return self.m1(x), self.m2(x)

def _pooler(x, rois):
  x = _roi_pooling(x, rois, size=(7,7), spatial_scale=1.0/16.0)
  return x.view(x.size(0), -1)

class _RPNClassifier(nn.Container):
  def __init__(self, n):
    super(_RPNClassifier, self).__init__()
    self.m1 = nn.Conv2d(n, 18, 3, 1, 1)
    self.m2 = nn.Conv2d(n, 36, 3, 1, 1)

  def forward(self, x):
    return self.m1(x), self.m2(x)

def model():
  _features = _Features()
  _classifier = _Classifier()
  _rpn_classifier = _RPNClassifier(3)

  _rpn = _RPN(
    classifier=_rpn_classifier
  )
  _model = _FasterRCNN(
    features=_features,
    pooler=_pooler,
    classifier=_classifier,
    rpn=_rpn
  )
  return _model

In [2]:
net = model()

C:\Users\Max\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\container.py:16: UserWarning: nn.Container is deprecated. All of it's functionality is now implemented in nn.Module. Subclass that instead.
  warnings.warn("nn.Container is deprecated. All of it's functionality "


In [3]:
net

FasterRCNN(
  (features): _Features(
    (m): Conv2d(3, 3, kernel_size=(3, 3), stride=(16, 16), padding=(1, 1))
  )
  (rpn): RPN(
    (rpn_classifier): _RPNClassifier(
      (m1): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (m2): Conv2d(3, 36, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (classifier): _Classifier(
    (m1): Linear(in_features=147, out_features=21, bias=True)
    (m2): Linear(in_features=147, out_features=84, bias=True)
  )
)